In [8]:
import sys
!{sys.executable} -m pip install requests beautifulsoup4 pandas selenium

c:\miniproject_1\.venv\Scripts\python.exe: No module named pip


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [10]:
def get_top_volume_stocks(limit=5):
    """네이버 금융에서 거래량 상위 종목을 수집하고 정제합니다."""
    url = "https://finance.naver.com/sise/sise_quant.naver"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 표(table)의 행(tr) 데이터 찾기
    table_rows = soup.select('table.type_2 tbody tr')
    
    stock_data = []
    
    for row in table_rows:
        cols = row.select('td')
        # 빈 줄(구분선) 제외 및 종목 데이터만 추출
        if len(cols) >= 10 and cols[1].text.strip(): 
            try:
                # 데이터 수집
                item_name = cols[1].text.strip()                     # 종목명
                target_url = cols[1].find('a')['href']
                item_code = target_url.split('code=')[-1]            # 종목코드 추출
                
                raw_price = cols[2].text.strip()                     # 현재가 (콤마 포함)
                raw_volume = cols[5].text.strip()                    # 거래량
                raw_per = cols[10].text.strip()                      # PER
                
                # --- [2] 데이터 구조화 및 정제 노력 ---
                # "2,300" -> 2300 정수형 변환 (콤마 제거)
                clean_price = int(raw_price.replace(',', ''))
                clean_volume = int(raw_volume.replace(',', ''))
                
                # PER 데이터가 "N/A"일 경우 예외 처리 후 float 변환
                clean_per = float(raw_per.replace(',', '')) if raw_per != 'N/A' else None
                
                stock_data.append({
                    '종목코드': item_code,
                    '종목명': item_name,
                    '현재가': clean_price,
                    '거래량': clean_volume,
                    'PER': clean_per
                })
                
                if len(stock_data) >= limit: # 지정한 개수만큼만 수집
                    break
                    
            except Exception as e:
                print(f"데이터 정제 중 오류 발생: {e}")
                continue
                
    # DataFrame으로 구조화
    df = pd.DataFrame(stock_data)
    return df

In [11]:
# [3] Selenium 활용: 특정 종목의 최신 뉴스 기사 동적 수집
# -----------------------------------------------------
def get_stock_news_selenium(item_code):
    """Selenium을 사용하여 종목의 동적 로딩 뉴스 제목을 수집합니다."""
    # Selenium 옵션 설정 (헤드리스 모드로 빠르고 조용하게 실행)
    options = Options()
    options.add_argument('--headless') # 브라우저 창 띄우지 않음
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    
    driver = webdriver.Chrome(options=options)
    news_titles = []
    
    try:
        # 네이버 금융 종목 뉴스 iframe 페이지 접속
        news_url = f"https://finance.naver.com/item/news_news.naver?code={item_code}&page=1"
        driver.get(news_url)
        
        # DOM이 렌더링되고 뉴스 테이블이 보일 때까지 최대 10초 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.type5.tbody_transparent'))
        )
        
        # 뉴스 제목 추출
        articles = driver.find_elements(By.CSS_SELECTOR, '.title a')
        for i, article in enumerate(articles):
            if i >= 3: break # 최신 기사 3개만 추출
            # 따옴표, 특수문자 등 정제
            clean_title = article.text.strip().replace("'", "").replace('"', '')
            news_titles.append(clean_title)
            
    except Exception as e:
        print(f"[{item_code}] 뉴스 수집 오류: {e}")
    finally:
        driver.quit() # 자원 반환
        
    return news_titles
# --- 메인 실행 코드 ---
if __name__ == "__main__":
    print("1. [requests/bs4] 거래량 상위 3개 종목 데이터 수집 및 정제 중...")
    top_stocks_df = get_top_volume_stocks(limit=3)
    
    # 뉴스 기사를 담을 파생 컬럼 생성 (구조화)
    top_stocks_df['최신뉴스'] = ""
    
    print("\n2. [selenium] 각 종목별 최신 뉴스 수집 중...")
    for idx, row in top_stocks_df.iterrows():
        code = row['종목코드']
        name = row['종목명']
        print(f" - {name}({code}) 뉴스 수집 중...")
        
        news_list = get_stock_news_selenium(code)
        
        # 리스트 형태의 뉴스를 하나의 문자열로 결합 (DB 저장을 위한 정제)
        top_stocks_df.at[idx, '최신뉴스'] = " | ".join(news_list)
        time.sleep(1) # 서버 부하 방지를 위한 1초 대기 (매너 타이머)
        
    print("\n✅ 최종 구조화 및 확인된 데이터프레임:")
    display(top_stocks_df) # 주피터 노트북 내장 함수로 예쁘게 테이블 출력

1. [requests/bs4] 거래량 상위 3개 종목 데이터 수집 및 정제 중...

2. [selenium] 각 종목별 최신 뉴스 수집 중...

✅ 최종 구조화 및 확인된 데이터프레임:


,최신뉴스
